In [1]:
import os
import sys
import shutil
import time
import pytz
import traceback

import Ska.engarchive.fetch as fetch
import Chandra.Time

import datetime as dt
import matplotlib.dates as mdate
from matplotlib import gridspec

import matplotlib.pyplot as plt
import mpld3

import numpy as np
import pandas as pd
from astropy.table import Table

from Scripts.msidlists import *
from Scripts.event_times import *
from Scripts.plot_stylers import *

from tqdm import tqdm as progressbar

from hrcsentinel import hrccore as hrc


# allow_subset=True should let us draw more data points
# fetch.data_source.set('maude allow_subset=True')
fetch.data_source.set('cxc', 'maude allow_subset=True')
# fetch.data_source.set('maude')

hrc.styleplots()
labelsizes = 15
plt.rcParams['axes.titlesize'] = labelsizes + 2
plt.rcParams['axes.labelsize'] = labelsizes
plt.rcParams['xtick.labelsize'] = labelsizes
plt.rcParams['ytick.labelsize'] = labelsizes

%config InlineBackend.figure_format = 'retina'

In [2]:
davec_temperature_msids

['2PMT1T',
 '2PMT2T',
 '2DTSTATT',
 '2DCENTRT',
 '2FHTRMZT',
 '2CHTRPZT',
 '2FRADPYT',
 '2CEAHVPT',
 '2CONDMXT',
 '2UVLSPXT']

In [3]:
fetch.data_source.set('cxc') # just use ska for this

In [4]:
def convert_to_doy(datetime_start):
    '''
    Return a string like '2020:237' that will be passed to start= in fetch.get_telem()
    '''

    year = datetime_start.year
    day_of_year = datetime_start.timetuple().tm_yday
    doystring = '{}:{}'.format(year, day_of_year)

    return doystring

In [5]:
data_start = convert_to_doy(oneday_pre_anomaly)
data_start

'2020:236'

lets go one 4 days post anomaly:

In [6]:
data_stop = '2020:241'

Fetch the data

In [7]:
dat = fetch.get_telem(davec_temperature_msids, start=data_start, stop=data_stop, sampling='full')

Fetching full-resolution data for MSIDS=['2PMT1T', '2PMT2T', '2DTSTATT', '2DCENTRT', '2FHTRMZT', '2CHTRPZT', '2FRADPYT', '2CEAHVPT', '2CONDMXT', '2UVLSPXT']
  from 2020:236:12:00:00.000 to 2020:241:12:00:00.000


In [8]:
dat

MSIDset([('2PMT1T',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2PMT2T',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2DTSTATT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2DCENTRT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2FHTRMZT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2CHTRPZT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2FRADPYT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 len=26342 dtype=float32 unit=DEGC>),
         ('2CEAHVPT',
          <MSID start=2020:236:12:00:00.000 stop=2020:241:12:00:00.000 l

In [9]:
def make_excel_datestring(date):
    date.replace(tzinfo=None)
    print(type(date))
    epoch = dt.datetime(1899, 12,30) # Excel epoch is 1900-01-00
    epoch.replace(tzinfo=None)
    delta = date - epoch
    return float(delta.days) + (float(delta.seconds) / 86400)

In [10]:
columns = [dat["2PMT1T"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2PMT1T"].times)),        
          dat["2PMT1T"].vals, 
          dat["2PMT2T"].vals,
          dat["2DTSTATT"].vals,
          dat["2DCENTRT"].vals,
          dat["2FHTRMZT"].vals,
          dat["2CHTRPZT"].vals,
          dat["2FRADPYT"].vals,
          dat["2CEAHVPT"].vals,
          dat["2CONDMXT"].vals,
          dat["2UVLSPXT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
               "PMT1 EED Temperature 2PMT1T (C)",
               "PMT2 EED Temperature 2PMT2T (C)",
               "OutDet1 Temperature 2DTSTATT (C)",
               "OutDet1 Temperature 2DCENTRT (C)",
               "FEA Box Temperature 2FHTRMZT (C)",
               "FEA Box Temperature 2CHTRPZT (C) (C)",
               "+Y EED Temperature 2FRADPYT (C)",
               "-Y EED Temperature 2CEAHVPT (C)",
               "Conduit Temperature 2CONDMXT (C)",
                "Snout Temperature 2UVLSPXT (C)"]

In [13]:
a = pd.DataFrame(data=columns).T
a.columns=column_names
a

,"Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",Human-readable Time,PMT1 EED Temperature 2PMT1T (C),PMT2 EED Temperature 2PMT2T (C),OutDet1 Temperature 2DTSTATT (C),OutDet1 Temperature 2DCENTRT (C),FEA Box Temperature 2FHTRMZT (C),FEA Box Temperature 2CHTRPZT (C) (C),+Y EED Temperature 2FRADPYT (C),-Y EED Temperature 2CEAHVPT (C),Conduit Temperature 2CONDMXT (C),Snout Temperature 2UVLSPXT (C)
0,7.14571e+08,2020-08-23 12:01:16.564149+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
1,7.14571e+08,2020-08-23 12:01:32.964150+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
2,7.14571e+08,2020-08-23 12:01:49.364152+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
3,7.14571e+08,2020-08-23 12:02:05.764153+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
4,7.14571e+08,2020-08-23 12:02:22.164155+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
5,7.14571e+08,2020-08-23 12:02:38.564157+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
6,7.14571e+08,2020-08-23 12:02:54.964158+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
7,7.14571e+08,2020-08-23 12:03:11.364160+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
8,7.14571e+08,2020-08-23 12:03:27.764161+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421
9,7.14571e+08,2020-08-23 12:03:44.164153+00:00,20.7229,24.0356,19.6715,18.6448,22.3485,22.9038,18.1404,26.3868,20.7229,12.9421


In [14]:
a.to_excel('temperatures_for_davec_FULL_RESOLUTION.xlsx')

In [13]:
eventdate

[datetime.datetime(2020, 8, 24, 3, 45, 54, 967615, tzinfo=<matplotlib.dates._UTC object at 0x7fb5c8d90518>)]

In [13]:
fetch.data_source.set('maude') # just use ska for this
hrc_i_turnon_msids = ['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_i_turnon_msids,start='2020:272', sampling='full')

Fetching full-resolution data for MSIDS=['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
  from 2020:272:12:00:00.000 to 2020:273:18:53:48.525


In [14]:
columns = [dat["2IMTPAST"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2IMTPAST"].times)),        
          dat["2IMTPAST"].vals, 
          dat["2IMBPAST"].vals,
          dat["2IMHBLV"].vals,
          dat["2LLDIALV"].vals,
          dat["2RSRFALV"].vals,
          dat["2GRDVALV"].vals,
          dat["2TLEV1RT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
                "2IMTPAST",
               "2IMBPAST",
               "2IMHBLV",
               "2LLDIALV",
               "2RSRFALV",
               "2GRDVALV",
               "2TLEV1RT"]

In [15]:
a = pd.DataFrame(data=columns).T
a.columns=column_names
a

,"Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",Human-readable Time,2IMTPAST,2IMBPAST,2IMHBLV,2LLDIALV,2RSRFALV,2GRDVALV,2TLEV1RT
0,7.17682e+08,2020-09-28 12:01:40.753999+00:00,0,0,128,132,185,142,0
1,7.17682e+08,2020-09-28 12:02:13.554003+00:00,0,0,128,132,185,142,0
2,7.17682e+08,2020-09-28 12:02:46.353996+00:00,0,0,128,132,185,142,0
3,7.17682e+08,2020-09-28 12:03:19.153999+00:00,0,0,128,132,185,142,0
4,7.17682e+08,2020-09-28 12:03:51.954002+00:00,0,0,128,132,185,142,0
5,7.17682e+08,2020-09-28 12:04:24.753995+00:00,0,0,128,132,185,142,0
6,7.17682e+08,2020-09-28 12:04:57.553998+00:00,0,0,128,132,185,142,0
7,7.17682e+08,2020-09-28 12:05:30.354001+00:00,0,0,128,132,185,142,0
8,7.17682e+08,2020-09-28 12:06:03.153994+00:00,0,0,128,132,185,142,0
9,7.17682e+08,2020-09-28 12:06:35.953997+00:00,0,0,128,132,185,142,0


In [16]:
a.to_excel('hrcI_turnon.xlsx')

In [ ]:
fetch.data_source.set('maude') 
hrc_i_turnon_msids = ['2IMTPAST', '2IMBPAST', '2IMHBLV', '2IMHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_i_turnon_msids,start='2020:272', sampling='full')

In [12]:
fetch.data_source.set('maude') 
hrc_s_turnon_msids = ['2SPTPAST', '2SPBPAST', '2SPHBLV', '2SPHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
dat = fetch.get_telem(hrc_s_turnon_msids,start='2020:299', sampling='full')

Fetching full-resolution data for MSIDS=['2SPTPAST', '2SPBPAST', '2SPHBLV', '2SPHVLV', '2LLDIALV', '2RSRFALV', '2GRDVALV', '2TLEV1RT']
  from 2020:299:12:00:00.000 to 2020:302:20:30:31.807


In [14]:
columns = [dat["2SPTPAST"].times, 
           mdate.num2date(hrc.convert_chandra_time(dat["2SPTPAST"].times)),        
          dat["2SPTPAST"].vals, 
          dat["2SPBPAST"].vals,
          dat["2SPHBLV"].vals,
          dat["2LLDIALV"].vals,
          dat["2RSRFALV"].vals,
          dat["2GRDVALV"].vals,
          dat["2TLEV1RT"].vals]

column_names = ["Chandra Time (seconds since 1998 Jan 1, 00:00:00 TT, or MJD 50814.0 TT)",
                "Human-readable Time",
                "2SPTPAST",
               "2SPBPAST",
               "2SPHBLV",
               "2LLDIALV",
               "2RSRFALV",
               "2GRDVALV",
               "2TLEV1RT"]

In [17]:
a = pd.DataFrame(data=columns).T
a.columns=column_names

In [16]:
a.to_excel('hrcS_turnon.xlsx')